In [1]:
import pandas as pd
import re
from typing import List

def extract_substrings(s: str) -> List[str]:
    # Define the regular expression pattern
    pattern = r'[A-Z]+_+[0-9]+'
    
    # Find all substrings that match the pattern
    matches = re.findall(pattern, s)
    
    # Return the list of matches
    return matches

# Example usage
example_string = "Space Invader PA__0290 & PA_0532, 113 rue Ober..."
result = extract_substrings(example_string)
print(result)  # Output: ['ABC_123', 'DEF_456', 'GHI__789', 'JKL____101112']


['PA__0290', 'PA_0532']


In [2]:
# Define a function to pad the number part of the code
def pad_number(match):
    code = match.group(0)
    parts = code.split('_')
    parts[1] = parts[1].zfill(4)
    return '_'.join(parts)


In [3]:
coord = pd.read_csv("data/20241111_umap_coord_layer.csv")
coord['code'] = coord['name'].map(extract_substrings)
coord = coord.explode("code")
coord['code'] = coord['code'].str.replace('_+','_', regex=True)
coord['code'] = coord['code'].str.replace(r'([A-Z]+_+[0-9]+)', pad_number, regex=True)
coord

,name,Latitude,Longitude,code
0,"Space Invader PA__0290 & PA_0532, 113 rue Ober...",48.865889,2.378150,PA_0290
0,"Space Invader PA__0290 & PA_0532, 113 rue Ober...",48.865889,2.378150,PA_0532
1,"Space Invader PA_0002, 45 rue Quincampoix",48.861071,2.350494,PA_0002
2,"Space Invader PA_0003, 14 rue de la Montagne S...",48.848851,2.348519,PA_0003
3,"Space Invader PA_0004, 22 rue Monge",48.847105,2.351085,PA_0004
...,...,...,...,...
1259,"Space Invader VRS_039, 0 rue de la Paroisse",48.808700,2.123890,VRS_0039
1260,"Space Invader VRS_040, 1 avenue de Maintenon",48.819504,2.137759,VRS_0040
1261,"Space Invader VRS_041, 42 rue du MarÃ©chal Foch",48.809835,2.134159,VRS_0041
1262,"Space Invader VRS_042, 54 avenue de Villeneuve...",48.818910,2.149957,VRS_0042


In [17]:
data = pd.read_excel("data/20241111_export_invader_spotter_art_listing.xlsx")
data = data[[0,1,3,4]].rename(columns={0:"photo_raw",1:"code_raw", 3:"etat_raw", 4:"date_raw"})
data['code'] = data['code_raw'].str.extract(r'([A-Z]+_+[0-9]+)', expand=False)
data['etat'] = data['etat_raw'].str.replace(r'<img[^>]*>', '', regex=True)
# Use str.replace with the padding function
data['code'] = data['code'].str.replace(r'([A-Z]+_+[0-9]+)', pad_number, regex=True)
print(data.shape[0], coord.shape[0])
data = pd.merge(data, coord, on="code",how="left")
print(data.shape[0])
data['photo'] = data['photo_raw'].str.extract(r'<img[^>]*src="([^"]+)"', expand=False)
data['icon'] = data['etat_raw'].str.extract(r'<img[^>]*src="([^"]+)"', expand=False)
data['photo'] = data['photo'].str.replace("grosplan/PA/","img/",regex=True)
# data['title'] = data['name'] + '<br/>' + data['etat']
data['etat_short'] = data['etat'].str.replace('Dernier état connu : ','',regex=True)
data['adresse'] = data['name'].str.replace("Space Invader PA_[0-9]+,","",regex=True)



1524 1287
1524


In [18]:

done = pd.read_csv("data/flashed.txt",header=None)
done = done.rename(columns={0:'code'})

data.loc[data['code'].isin(done['code']), "etat_short"] = "Flashé !"
data.loc[data['code'].isin(done['code']), "icon"] = "nav/spot_invader_flashed.png"

In [19]:
data[['etat_short']].value_counts().reset_index()

,etat_short,count
0,OK,856
1,Flashé !,275
2,Détruit !,254
3,Un peu dégradé,68
4,Dégradé,54
5,Très dégradé,11
6,Non visible,6


In [6]:
data

,photo_raw,code_raw,etat_raw,date_raw,code,etat,name,Latitude,Longitude,photo,icon,etat_short,adresse
0,"<img src=""grosplan/PA/PA_0002-grosplan.png"" ti...","<font class=""normal""><b>PA_02 [10 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : juin 2019 (spott),PA_0002,Dernier état connu : OK,"Space Invader PA_0002, 45 rue Quincampoix",48.861071,2.350494,img/PA_0002-grosplan.png,nav/spot_invader_ok.png,OK,45 rue Quincampoix
1,"<img src=""grosplan/PA/PA_0003-grosplan.png"" ti...","<font class=""normal""><b>PA_03 [10 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : octobre 2019 (report),PA_0003,Dernier état connu : OK,"Space Invader PA_0003, 14 rue de la Montagne S...",48.848851,2.348519,img/PA_0003-grosplan.png,nav/spot_invader_flashed.png,Flashé !,14 rue de la Montagne Sainte-GeneviÃ¨ve
2,"<img src=""grosplan/PA/PA_0004-grosplan.png"" ti...","<font class=""normal""><b>PA_04 [10 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : mai 2022,PA_0004,Dernier état connu : Dégradé,"Space Invader PA_0004, 22 rue Monge",48.847105,2.351085,img/PA_0004-grosplan.png,nav/spot_invader_flashed.png,Flashé !,22 rue Monge
3,"<img src=""grosplan/PA/PA_0005-grosplan.png"" ti...","<font class=""normal""><b>PA_05 [10 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : novembre 2020 (Live FlashInva...,PA_0005,Dernier état connu : Dégradé,"Space Invader PA_0005, 25 rue Guisarde",48.851810,2.333760,img/PA_0005-grosplan.png,nav/spot_invader_flashed.png,Flashé !,25 rue Guisarde
4,"<img src=""grosplan/PA/PA_0006-grosplan.png"" ti...","<font class=""normal""><b>PA_06 [10 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : juillet 2022,PA_0006,Dernier état connu : Très dégradé,"Space Invader PA_0006, 40 rue de Seine",48.855063,2.336769,img/PA_0006-grosplan.png,nav/spot_invader_destroyed.png,Très dégradé,40 rue de Seine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,"<img src=""grosplan/PA/PA_1430-grosplan.png"" ti...","<font class=""normal""><b>PA_1430 [50 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : juin 2022,PA_1430,Dernier état connu : OK,"Space Invader PA_1430, 130 boulevard Berthier",48.888272,2.299683,img/PA_1430-grosplan.png,nav/spot_invader_ok.png,OK,130 boulevard Berthier
1251,"<img src=""grosplan/PA/PA_1431-grosplan.png"" ti...","<font class=""normal""><b>PA_1431 [100 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : novembre 2019 (report),PA_1431,Dernier état connu : OK,"Space Invader PA_1431, dernier Ã©tage de la To...",48.858271,2.294608,img/PA_1431-grosplan.png,nav/spot_invader_ok.png,OK,dernier Ã©tage de la Tour Eiffel
1252,"<img src=""grosplan/PA/PA_1432-grosplan.png"" ti...","<font class=""normal""><b>PA_1432 [100 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : décembre 2019 (report),PA_1432,Dernier état connu : OK,"Space Invader PA_1432, place Igor Stravinsky",48.859530,2.351160,img/PA_1432-grosplan.png,nav/spot_invader_flashed.png,Flashé !,place Igor Stravinsky
1253,"<img src=""grosplan/PA/PA_1433-grosplan.png"" ti...","<font class=""normal""><b>PA_1433 [20 pts]</b>","Dernier état connu : <img src=""nav/spot_invade...",Date et source : mai 2022,PA_1433,Dernier état connu : OK,"Space Invader PA_1433, 36 rue Daubenton",48.841104,2.352405,img/PA_1433-grosplan.png,nav/spot_invader_flashed.png,Flashé !,36 rue Daubenton


In [20]:
print(data.shape[0])
hors_carte = data[data['Latitude'].isna()]
print(hors_carte.shape[0])
hors_carte = hors_carte[hors_carte['etat_short']!=" Détruit !"]
print(hors_carte.shape[0])
hors_carte = hors_carte[hors_carte['etat_short']!="Flashé !"]
print(hors_carte.shape[0])

1524
269
178
159


In [ ]:
# hors_carte.to_excel("data/invaders_todo_hors_carte.xlsx")

In [ ]:
# > recherche sur google "pa_xxx invader"
# > analyse résultats sur flickr par exemple
# > recherche de l'adresse sur Google
# > récup coordonnées sur Google Maps (conversion . en ,)